In [ ]:
import os
from os.path import join
import shutil

paths = ['continued/sensitive', 'general']
os.makedirs('single_characters', exist_ok=True)

i=0
files = []
for path in paths:
    for file in os.listdir(path):
        if file.endswith('.jpg'):
            if file.replace('.jpg', '.txt') in os.listdir(path):
                files.append(file.replace('.jpg', ''))

        i+=1
        print(i, end = '\r')

print(len(files))

i = 0
for path in paths:
    for file in os.listdir(path):
        if file.replace('.jpg', '').replace('.txt', '') in files:
            shutil.copy(join(path, file), join('single_characters', file))

        i+=1
        print(i, end = '\r')


len(os.listdir('single_characters'))


In [ ]:
import re
import shutil
import os
from os.path import join


path = 'single_characters'

def contains_single_character(danbooru_string):
    # Patterns indicating multiple characters
    multiple_patterns = [
        r'\b\+girls\b',
        r'\b\+boys\b',
        r'\b\+others\b',
        r'\bmultiple girls\b',
        r'\bmultiple boys\b',
        r'\b\d\+girls\b',
        r'\b\d\+boys\b',           
        r'\b\d\+others\b'
        r'\bgroup\b',
        r'\bsiblings\b',
        r'\bbrothers\b',
        r'\bsisters\b',
        r'\bbrothers and sisters\b',
    ]
    
    single_patterns = [
        r'\b1girl\b',
        r'\b1boy\b',
        r'\b1other\b',
        r'\bsolo\b',
    ]

    # Check for matches in multiple-character patterns
    for pattern in multiple_patterns:
        if re.search(pattern, danbooru_string):
            return False
        
    # Count occurrences of "1boy" and "1girl"
    single_counts = sum(bool(re.search(pattern, danbooru_string)) for pattern in single_patterns[:-1])  # Exclude "solo"
    if single_counts > 1:  # If more than one single character term exists
        return False

    # If "solo" is present, treat as a single character
    if re.search(r'\bsolo\b', danbooru_string):
        return True

    # Check for matches in single-character patterns
    for pattern in single_patterns:
        if re.search(pattern, danbooru_string):
            return True

   
    return True

single_files = {}
from tqdm import tqdm

for i,file in enumerate(tqdm(os.listdir(path))):
    if file.endswith('.txt'):
        with open(join(path, file), 'r') as f:
            content = f.read()
        if contains_single_character(content):
            single_files[file.replace('.txt', '')] = content
            
for file in tqdm(os.listdir(path)):
    if file.split('.')[0] not in single_files.keys():
        os.remove(os.path.join(path, file))


In [ ]:
len(os.listdir('single_characters'))

In [ ]:
# some checks

def has_only_one_number(string):
    # Find all digits in the string
    digits = re.findall(r'\d', string)
    # Check if there is exactly one digit
    return len(digits) != 1

# Process files to check for strings with only one number
files_with_one_number = {}

for file in tqdm(os.listdir(path)):
    if file.endswith('.txt'):
        with open(join(path, file), 'r') as f:
            content = f.read()
        if has_only_one_number(content):
            files_with_one_number[file.replace('.txt', '')] = content



In [1]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import re
import json
torch.random.manual_seed(0)

model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3.5-mini-instruct", 
    device_map="cuda", 
    torch_dtype="auto", 
    trust_remote_code=True, 
)
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3.5-mini-instruct", padding_side = 'left')

def get_json(text):
    match = re.search(r"\{.*?\}", text, re.DOTALL)
    if match:
        extracted_json = match.group()
        # Convert to JSON object to verify
        json_data = json.loads(extracted_json)
        return json_data
    else:
        print("No JSON found in the text.")

`flash-attention` package not found, consider installing for better performance: No module named 'flash_attn'.
Current `flash-attention` does not support `window_size`. Either upgrade or use `attn_implementation='eager'`.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [7]:

example_string ="1girl, kousaka kirino, ore no imouto ga konna ni kawaii wake ga nai, itou ryuusei, ||| sensitive, crossed legs, feet, kousaka kirino's school uniform, legs, long hair, no shoes, school uniform, serafuku, sitting, socks, solo, oldest, good quality, very aesthetic"
main_string = "1girl, akiyama mio, k-on!, mifumi takafumi, ||| sensitive, animal ears, black eyes, black hair, cat ears, kneeling, long hair, sakuragaoka high school uniform, school uniform, shoes, solo, uwabaki, oldest, normal quality, very aesthetic" 
# main_string = """1girl, hirasawa yui, k-on!, shiou tsuyukusa, ||| sensitive, brown eyes, brown hair, checkered, don't say "lazy", dress, foreshortening, frills, guitar, hair ornament, instrument, pantyhose, short hair, solo, striped clothes, striped pantyhose, vertical-striped clothes, vertical-striped pantyhose, oldest, normal quality, very aesthetic"""
messages = [
    {"role": "system", "content": "You are a helpful information extractor from danbooru strings"},
    {"role": "user", "content": f"""{example_string}

            Can you get me the following attributes in JSON format from this danbooru. Make sure all the attributes are a string. Only output the json. All the values should be 1-2 words


            Age
            Gender
            Ethnicity
            Hair Style (Straight, Curly)
            Hair Color 
            Hair Length (Short, Medium, Long)
            Eye Color
            Body Type (Athletic, Curvy, Thin)
            Dress (Only the type should be mentioned)"""},
    {"role": "assistant", "content": """{
        "Age": "Teen",
        "Gender": "Female",
        "Ethnicity": "Japanese",
        "Hair Style": "Straight",
        "Hair Color": "Blonde",
        "Hair Length": "Long",
        "Eye Color": "Blue",
        "Body Type": "Slim",
        "Dress": "Serafuku"
        }"""},
    {"role": "user", "content": f"""{main_string}

            Can you get me the following attributes in JSON format from this danbooru. Make sure all the attributes are a string. Only output the json. All the values should be 1-2 words


            Age
            Gender
            Ethnicity
            Hair Style (Straight, Curly)
            Hair Color 
            Hair Length (Short, Medium, Long)
            Eye Color
            Body Type (Athletic, Curvy, Thin)
            Dress (Only the type should be mentioned)"""},
]

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
)

generation_args = {
    "max_new_tokens": 150,
    "return_full_text": False,
    "temperature": 0.0,
    "do_sample": False,
}



In [8]:
output = pipe([messages], **generation_args)

/raid/biplab/sarthak/miniconda3/envs/flsetup/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:590: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


In [9]:
output[0][0]

{'generated_text': ' {\n        "Age": "Teen",\n        "Gender": "Female",\n        "Ethnicity": "Japanese",\n        "Hair Style": "Straight",\n        "Hair Color": "Black",\n        "Hair Length": "Long",\n        "Eye Color": "Black",\n        "Body Type": "Slim",\n        "Dress": "School Uniform"\n        }\n\n\nPlease note that the provided text does not contain explicit information about the age, ethnicity, hair style, hair color, eye color, and body type. Therefore, I have used the most likely assumptions based on the context'}

In [10]:
get_json(output[0][0]['generated_text'])

{'Age': 'Teen',
 'Gender': 'Female',
 'Ethnicity': 'Japanese',
 'Hair Style': 'Straight',
 'Hair Color': 'Black',
 'Hair Length': 'Long',
 'Eye Color': 'Black',
 'Body Type': 'Slim',
 'Dress': 'School Uniform'}

## Converting into a dataset for pretraining

In [83]:
import pandas as pd
import json

data_list = []
for chunk_idx in range(2):  # Assuming num_chunks is defined elsewhere
    with open(f'generated_outputs_chunk_{chunk_idx}.jsonl', 'r') as file:
        for l in file.readlines():
            content = json.loads(l)
            key = list(content.keys())[0]
            value = content[key]
            try:
                data_list.append(pd.DataFrame(data = value, index=[key]))
            except:
                pass
            # data_list.append(key)

df = pd.concat(data_list)
columns = list(df.columns)
columns_lower = ['age', 'gender', 'ethnicity', 'hair style', 'hair color', 'hair length', 'eye color', 'body type', 'dress']
df.drop(columns=[x for x in columns if x.lower() not in columns_lower], inplace=True)

In [84]:
unique_counts = {}
for column in df.columns:
    counts = df[column].value_counts()
    # Remove values with a low count (e.g., less than 20)
    counts = counts[counts >= 20]
    unique_counts[column] = counts.to_dict()


df_copy = df.copy()
for column in unique_counts.keys():
    df_copy = df_copy[df_copy[column].isin(unique_counts[column])]

df = df_copy
df.head()

,Age,Gender,Ethnicity,Hair Style,Hair Color,Hair Length,Eye Color,Body Type,Dress
danbooru_1033555_1d84ae2b33b634b8cac9e6e9e8892a14.txt_0,Early,Female,Unknown,Straight,Black,Long,Brown,Slim,Sundress
danbooru_1033683_97ed963564accf59ed05a651151d5a12.txt_0,Early,Female,Japanese,Long,Black,Long,Brown,Slim,Serafuku
danbooru_1079117_daf0cefc12ddc98f3309bf9b409c4934.txt_0,Early,Female,Japanese,Long,Black,Long,Black,Slim,School Uniform
danbooru_1079118_caaae512e97c073b8480451a4f4eeef4.txt_0,Early,Female,Japanese,Long,Black,Long,Blue,Slim,None
danbooru_1079184_5e964b47430f6c3eb0e335996482d975.txt_0,Early,Female,Japanese,Long,Black,Long,Brown,Slim,School Uniform


In [85]:
encodings = {column: {value: idx for idx, value in enumerate(sorted(unique_counts[column].keys()))} for column in unique_counts.keys()}
for column in encodings.keys():
    df[column] = df[column].map(encodings[column])

import json

with open('encodings.json', 'w') as json_file:
    json.dump(encodings, json_file)

train_df = df.sample(frac=0.9, random_state=42)  # 80% for training
test_df = df.drop(train_df.index)  # Remaining 20% for testing

train_df.reset_index(inplace=True)
test_df.reset_index(inplace=True)
train_df.to_csv('train_data.csv', index=False)
test_df.to_csv('test_data.csv', index=False)

train_df

,index,Age,Gender,Ethnicity,Hair Style,Hair Color,Hair Length,Eye Color,Body Type,Dress
0,danbooru_1329597_12e81cb275df88fcc46d80ccf9e20...,2,0,3,4,1,0,2,7,26
1,danbooru_5846809_d40226fbdcd41f00ccc74d8f23efc...,7,1,5,10,0,2,5,0,12
2,danbooru_2719090_f6240a50566c5bd0c596d23063a5c...,3,0,3,4,1,3,2,8,20
3,danbooru_1204804_dfa27a941a4bc4d783ac80d4a3616...,7,0,3,4,0,0,11,7,13
4,danbooru_2758066_02e53aeb7fb9e23ffb9bf112df714...,3,0,5,4,5,3,2,9,20
...,...,...,...,...,...,...,...,...,...,...
2213,danbooru_4344758_3d3205cb8e248e098a829c2bd3e61...,7,1,3,10,1,2,2,7,25
2214,danbooru_4201058_c0a285731616cc34c023d931d6662...,4,0,3,4,3,0,3,7,23
2215,danbooru_472675_9434e6644b6b71135942d0f749617e...,5,0,3,12,3,0,3,4,13
2216,danbooru_5817460_4fe517e056adfc20b6da19c39415e...,7,1,5,1,0,0,1,3,18
